In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.python.framework import ops
import tensorflow as tf
import numpy as np

2023-12-07 14:04:43.249589: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 14:04:43.389100: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 14:04:43.389129: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 14:04:43.389929: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 14:04:43.470984: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 14:04:43.472358: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf = tf.compat.v1
tf.disable_v2_behavior()

In [3]:
sess = tf.Session()

2023-12-07 14:04:46.680602: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-07 14:04:46.777715: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [5]:
#We then normalize the train and test set features 
# Z- scoring  or Gaussian Normalization
X_train = X_train - np.mean(X_train) / X_train.std()
X_test = X_test - np.mean(X_test) / X_test.std()

In [6]:
# Convert labels into one-hot encoded vectors
train_labels = y_train
test_labels = y_test

In [7]:
num_class = 10

In [8]:
# Set model parameters
batch_size = 784
samples =500
learning_rate = 0.03
img_width = X_train[0].shape[0]
img_height = X_train[0].shape[1]
target_size = max(train_labels) + 1
num_channels = 1 # greyscale = 1 channel
epoch = 3
no_channels = 1
conv1_features = 30
filt1_features = 5
conv2_features = 15
filt2_features = 3
max_pool_size1 = 2 # NxN window for 1st max pool layer
max_pool_size2 = 2 # NxN window for 2nd max pool layer
fully_connected_size1 = 150

In [9]:
# Declare model placeholders
x_input_shape = (batch_size, img_width, img_height, num_channels)
x_input = tf.placeholder(tf.float32, shape=x_input_shape)
y_target = tf.placeholder(tf.int32, shape=(batch_size))
eval_input_shape = (samples, img_width, img_height, num_channels)
eval_input = tf.placeholder(tf.float32, shape=eval_input_shape)
eval_target = tf.placeholder(tf.int32, shape=(samples))

In [10]:
# Declare model variables
W1 = tf.Variable(tf.random_normal([filt1_features, filt1_features, no_channels, conv1_features]))
b1 = tf.Variable(tf.ones([conv1_features]))
W2 = tf.Variable(tf.random_normal([filt2_features, filt2_features, conv1_features, conv2_features]))
b2 = tf.Variable(tf.ones([conv2_features]))

In [11]:
# Declare model variables for fully connected layers
resulting_width = img_width // (max_pool_size1 * max_pool_size2)
resulting_height = img_height // (max_pool_size1 * max_pool_size2)
full1_input_size = resulting_width * resulting_height * conv2_features
W3 = tf.Variable(tf.truncated_normal([full1_input_size, fully_connected_size1],
                          stddev=0.1, dtype=tf.float32))
b3 = tf.Variable(tf.truncated_normal([fully_connected_size1], stddev=0.1, dtype=tf.float32))
W_out = tf.Variable(tf.truncated_normal([fully_connected_size1, target_size],
                                               stddev=0.1, dtype=tf.float32))
b_out = tf.Variable(tf.truncated_normal([target_size], stddev=0.1, dtype=tf.float32))

In [12]:
# Define helper functions for the convolution and maxpool layers:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.elu(conv_with_b)
    return conv_out

In [13]:
def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [14]:
# Initialize Model Operations
def my_conv_net(input_data):
    conv_out1 = conv_layer(input_data, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    # Second Conv-ReLU-MaxPool Layer
    conv_out2 = conv_layer(maxpool_out1, W2, b2)
    maxpool_out2 = maxpool_layer(conv_out2)

    # Transform Output into a 1xN layer for next fully connected layer
    final_conv_shape = maxpool_out2.get_shape().as_list()
    final_shape = final_conv_shape[1] * final_conv_shape[2] * final_conv_shape[3]
    flat_output = tf.reshape(maxpool_out2, [final_conv_shape[0], final_shape])

    # First Fully Connected Layer
    fully_connected1 = tf.nn.relu(tf.add(tf.matmul(flat_output, W3), b3))
    # Second Fully Connected Layer
    final_model_output = tf.add(tf.matmul(fully_connected1, W_out), b_out)

    return final_model_output

In [16]:
model_output = my_conv_net(x_input)
test_model_output = my_conv_net(eval_input)

In [17]:
# Declare Loss Function (softmax cross entropy)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model_output, labels=y_target))

In [18]:
# Create a prediction function
prediction = tf.nn.softmax(model_output)
test_prediction = tf.nn.softmax(test_model_output)

In [19]:
# Create accuracy function
def get_accuracy(logits, targets):
    batch_predictions = np.argmax(logits, axis=1)
    num_correct = np.sum(np.equal(batch_predictions, targets))
    return(100. * num_correct/batch_predictions.shape[0])

In [20]:
# Create an optimizer
my_optimizer = tf.train.AdamOptimizer(learning_rate, 0.9)
train_step = my_optimizer.minimize(loss)

# Initialize Variables
varInit = tf.global_variables_initializer()
sess.run(varInit)

2023-12-07 14:09:04.044292: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled


In [21]:
# Start training loop
train_loss = []
train_acc = []
test_acc = []

for i in range(epoch):
    random_index = np.random.choice(len(X_train), size=batch_size)
    random_x = X_train[random_index]
    random_x = np.expand_dims(random_x, 3)
    random_y = train_labels[random_index]   
     
    train_dict = {x_input: random_x, y_target: random_y}
    
    sess.run(train_step, feed_dict=train_dict)
    temp_train_loss, temp_train_preds = sess.run([loss, prediction], feed_dict=train_dict)
    temp_train_acc = get_accuracy(temp_train_preds, random_y)
    
    
    eval_index = np.random.choice(len(X_test), size=500)
    eval_x = X_test[eval_index]
    eval_x = np.expand_dims(eval_x, 3)
    eval_y = test_labels[eval_index]
    test_dict = {eval_input: eval_x, eval_target: eval_y}
    test_preds = sess.run(test_prediction, feed_dict=test_dict)
    temp_test_acc = get_accuracy(test_preds, eval_y)
    # Record and print results
    train_loss.append(temp_train_loss)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    acc_and_loss = [(i+1), temp_train_loss, temp_train_acc, temp_test_acc]
    acc_and_loss = [np.round(x,2) for x in acc_and_loss]
    print('Epoch # {}. Train Loss: {:.2f}. Train Acc : {:.2f} . temp_test_acc : {:.2f}'.format(i+1,temp_train_loss,temp_train_acc,temp_test_acc))

2023-12-07 14:09:15.388154: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 73758720 exceeds 10% of free system memory.
2023-12-07 14:09:15.441229: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27941760 exceeds 10% of free system memory.
2023-12-07 14:09:15.441637: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27941760 exceeds 10% of free system memory.
2023-12-07 14:09:15.485748: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 36879360 exceeds 10% of free system memory.
2023-12-07 14:09:15.485809: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 73758720 exceeds 10% of free system memory.


Epoch # 1. Train Loss: 136220.16. Train Acc : 9.95 . temp_test_acc : 9.60
Epoch # 2. Train Loss: 79646.30. Train Acc : 15.18 . temp_test_acc : 15.80
Epoch # 3. Train Loss: 74905.19. Train Acc : 10.33 . temp_test_acc : 10.00
